# Doc file tu fol /processed_data
# Duyet tung dong -> chuyen than vector 
# Luu voi format:

{
  "chunks": [
    {
      chunk_index: 1,
      vector: [0.0910910239]
    }
  ],
  // lUU CHO KHAC ... TOI UU. HOAC LA LUNG CHUNG LUON .
  "metadata": { id, title, author, tags, publish_time...}
}

# Seach duoc - (Anh + Huy)

In [4]:
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
model = AutoModel.from_pretrained("vinai/phobert-base")


# Test
# if __name__ == '__main__':
#     sample = "Việt Nam vô địch AFF Cup"
#     vec = vectorize(sample)
#     print(f"Vector result: {vec}")


/mnt/dae4cf6d-35d6-4a5e-8ca3-b5ad6ef52795/py_env_research/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import pandas as pd

# Đường dẫn file dữ liệu gốc và file đầu ra
BASE_OUTPUT_DATA_DIR = "../data/processed_data/Original_news_dataset.csv"

# Đọc dữ liệu từ file CSV
df = pd.read_csv(BASE_OUTPUT_DATA_DIR, dtype=str, nrows=2000)
df.head()

,Unnamed: 0,id,author,content,picture_count,processed,source,title,topic,url,crawled_at
0,0,218270,NaN,"chiều 31/7 , công_an tỉnh thừa_thiên - huế đã ...",3,0,docbao.vn,"Tên cướp tiệm vàng tại Huế là đại uý công an, ...",Pháp luật,https://docbao.vn/phap-luat/ten-cuop-tiem-vang...,2022-08-01 09:09:22.817308
1,1,218269,(Nguồn: Sina),"gần đây , thứ_trưởng bộ phát_triển kỹ_thuật_số...",1,0,vtc.vn,"Bỏ qua mạng 5G, Nga tiến thẳng từ 4G lên 6G",Sống kết nối,https://vtc.vn/bo-qua-mang-5g-nga-tien-thang-t...,2022-08-01 09:09:21.181469
2,2,218268,Hồ Sỹ Anh,kết_quả thi tốt_nghiệp thpt năm 2022 cho thấy ...,3,0,thanhnien.vn,Địa phương nào đứng đầu cả nước tổng điểm 3 mô...,Giáo dục,https://thanhnien.vn/dia-phuong-nao-dung-dau-c...,2022-08-01 09:09:15.311901
3,3,218267,Ngọc Ánh,thống_đốc kentucky andy beshear hôm 31/7 cho_h...,1,0,vnexpress,Người chết trong mưa lũ 'nghìn năm có một' ở M...,Thế giới,https://vnexpress.net/nguoi-chet-trong-mua-lu-...,2022-08-01 09:09:02.211498
4,4,218266,HẢI YẾN - MINH LÝ,vụ tai_nạn giao_thông liên_hoàn trên phố đi bộ...,12,0,soha,"Hải Phòng: Hình ảnh xe ""điên"" gây tai nạn liên...",Thời sự - Xã hội,https://soha.vn/hai-phong-hinh-anh-xe-dien-gay...,2022-08-01 09:09:01.601170


In [6]:
def merge_sentences(sentences, max_tokens=256):
  chunks = []
  current_chunk = []
  current_length = 0

  for sentence in sentences:
    if not sentence:
      continue
    tokenized = tokenizer.tokenize(sentence)
    token_length = len(tokenized)

    if token_length >= max_tokens:
      # Nếu câu này đã >= max_tokens thì tách riêng, không ghép thêm
      if current_chunk:
        chunks.append(" ".join(current_chunk))
        current_chunk = []
        current_length = 0
      chunks.append(sentence)
    else:
      if current_length + token_length <= max_tokens:
        current_chunk.append(sentence)
        current_length += token_length
      else:
        if current_chunk:
          chunks.append(" ".join(current_chunk))
        current_chunk = [sentence]
        current_length = token_length

  if current_chunk:
    chunks.append(" ".join(current_chunk))

  return chunks



In [25]:
# Chuyen content -> vector
from underthesea import sent_tokenize

def vectorize(text):
    tokens = tokenizer(text, return_tensors='pt',  
                       truncation=True, padding='max_length', max_length=258)
    with torch.no_grad(): 
      
        outputs = model(**tokens)
    return outputs.last_hidden_state.mean(dim=1).squeeze().tolist()

def process_content(sentences):
    # sentences = sent_tokenize(content
    # print(sentences)
    chunks = merge_sentences(sentences)
    vectors = [vectorize(chunk) for chunk in chunks]
    # vectors = [vectorize(chunk) for chunk in sentences]
    return vectors
# fj

output_data = []

for idx, row in df.head(10).iterrows():
  content = row['content']
  if pd.isnull(content):
    continue
  
  ## 
  sentences = content.split('.')
  vectors = process_content(sentences)
  
  chunks = []
  # Bien doi thanh JSON
  for i, vector in enumerate(vectors):
    chunks.append({
      "chunk_index": i + 1,
      "vector": vector,
      "text": sentences[i] if i < len(sentences) else ""
    })
  metadata = {
    "id": row['id'],
    "title": row['title'],
    "author": row['author'],
    "tags": row['topic'],
    "publish_time": row['crawled_at'],
    "source": row['source'],
    "url": row['url']
  }
  output_data.append({
    "chunks": chunks,
    "metadata": metadata
  })

In [27]:
import json
with open("output_vectors.json", "w", encoding="utf-8") as f:
  json.dump(output_data, f, ensure_ascii=False, indent=2)


In [ ]:
!pip3 install qdrant_client

  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 16.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.8 MB/s eta 0:00:00
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16/16 [qdrant_client]0m [qdrant_client]


In [45]:
from uuid import uuid4
from qdrant_client import QdrantClient, models


# Use in-memory Qdrant instance
client = QdrantClient(":memory:")

collection_name = "news_vectors_v4"

# Tạo collection nếu chưa có
if collection_name not in [c.name for c in client.get_collections().collections]:
  client.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
      size=len(output_data[0]['chunks'][0]['vector']),
      distance="Cosine",
    )
  )

# Lưu vectors vào Qdrant
for item in output_data:
  points = []
  for chunk in item["chunks"]:
    points.append(
      models.PointStruct(
        id=str(uuid4()),
        vector=chunk["vector"],
        payload={
          "text": chunk["text"],
          **item["metadata"]
        }
      )
    )
  client.upsert(
    collection_name=collection_name,
    points=points,
    
  )


/tmp/ipykernel_114990/3042891842.py:12: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


In [59]:
from underthesea import word_tokenize
from qdrant_client.models import Filter, FieldCondition, MatchValue


user_query = word_tokenize("Binh Duong co tong 3 mon toan".strip().lower(), format="text")
print(f"User query: {user_query}")
user_query_vector = vectorize(user_query)
query_result = client.query_points(
    collection_name=collection_name,
    query=user_query_vector,
    limit=3, 
    with_payload=True,
)

query_result.dict()

User query: binh_duong co_tong 3 mon_toan


/tmp/ipykernel_114990/991084370.py:15: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  query_result.dict()


{'points': [{'id': 'a7e771c1-0389-43e3-b5e2-3b783a0bb259',
   'version': 0,
   'score': 0.6505497047340189,
   'payload': {'text': ' ronaldo là cầu_thủ duy_nhất bị ten hag rút ra ngay sau giờ nghỉ , nhường chỗ cho amad diallo ',
    'id': '218261',
    'title': 'Bị thay sau hiệp 1, Ronaldo đùng đùng bỏ về sớm',
    'author': 'Đỗ Trung',
    'tags': 'Ngoại hạng Anh',
    'publish_time': '2022-08-01 09:08:42.496092',
    'source': 'bongdaplus',
    'url': 'https://bongdaplus.vn/ngoai-hang-anh/ronaldo-dung-dung-bo-ve-som-3726962208.html'},
   'vector': None,
   'shard_key': None,
   'order_value': None},
  {'id': '1bad41ad-3e84-45e1-8b52-d6cd9d781374',
   'version': 0,
   'score': 0.5914473290531295,
   'payload': {'text': '',
    'id': '218268',
    'title': 'Địa phương nào đứng đầu cả nước tổng điểm 3 môn văn, toán, ngoại ngữ?',
    'author': 'Hồ Sỹ Anh',
    'tags': 'Giáo dục',
    'publish_time': '2022-08-01 09:09:15.311901',
    'source': 'thanhnien.vn',
    'url': 'https://thanhnien